In [254]:
from scipy.stats import permutation_test
from itertools import groupby
import numpy as np

In [255]:
all_events = []
with open('exp_arppu_problem.tsv', 'r') as input:
    input.readline()
    for line in input:
        parts = line.strip().split("\t")
        userid = parts[0]
        timestamp = int(parts[1])
        action = parts[2]
        value = float(parts[3])
        testids = parts[4]
        all_events.append((userid, timestamp, action, value, testids))

all_events.sort(key=lambda x: x[0])

## Интерпретатор результата перестановочного теста

Уровень значимости $\alpha$ всегда будет $0.05$

In [291]:
alpha = 0.05
def print_result(result):
    print(result.pvalue, end=' ')
    if result.pvalue > alpha:
        print('gray')
    elif result.statistic > 0:
        print('green')
    else:
        print('red')

## Разбиение на выборки

In [257]:
exp_testid = 34265
control_testid = 34266

def with_testids(testid):
    return lambda e: str(testid) in e[4]

exp_events = list(filter(with_testids(exp_testid), all_events))
control_events = list(filter(with_testids(control_testid), all_events))

## Прокраска метрик ARPU и ARPPU 

In [258]:
def revenues_by_user(events):
    return [
    sum(e[3] for e in user_events if e[2] == "confirmation")
    for user, user_events in groupby(events, lambda e: e[0])
    ]
exp_revenues = revenues_by_user(exp_events)
control_revenues = revenues_by_user(control_events)

In [267]:
def arpu(a):
    return np.mean(a)
def statistic(a, b):
    return arpu(a) - arpu(b)

np.random.seed(177)
result = permutation_test((exp_revenues, control_revenues), statistic, permutation_type="independent")

print("RESULTS for metric ARPU: ", end='')
print_result(result)

RESUTLS for metric ARPU: 0.001 red


In [268]:
def is_pay(revenue):
    return revenue > 0
def arppu(a):
    return np.mean(list(filter(is_pay, a)))
def statistic_p(a, b):
    return arppu(a) - arppu(b) 

np.random.seed(177)
result = permutation_test((exp_revenues, control_revenues), statistic_p, permutation_type="independent")

print("RESULTS for metric ARPPU: ", end='')
print_result(result)

RESUTLS for metric ARPPU: 0.042 green


## Гипотеза №1: Увеличился средний чек заказа пользователя

$H_{0}$ – средний чек заказа пользователя не изменился.
$H_{A}$ – средний чек заказа изменился. 

In [295]:
def receipts_by_user(events):
    receipts = [e[3] for e in events if e[2] == "confirmation"]
    return receipts
exp_receipts = receipts_by_user(exp_events)
control_receipts = receipts_by_user(control_events)

In [300]:
def effect(a):
    return np.mean(a)
def statistic(a, b):
    return effect(a) - effect(b) 

result = permutation_test((exp_receipts, control_receipts), statistic, permutation_type="independent")

print("RESULTS for metric \"Average order receipt of the user\": ", end='')
print_result(result)

RESULTS for metric "Average order receipt of the user": 0.0002 green


Эффект является статистически значимым, поэтому возможно опровергнуть нулевую гипотезу.

Средний чек заказа пользователя действительно увеличился, о чем свидетельствует "green" в результатах теста.

## Гипотеза №2: Уменьшилось количество платящих пользователей

$H_{0}$ – количество платящих пользователей не изменилось.
$H_{A}$ – количество платящих пользователей изменилось. 

In [304]:
def amount_of_paying_users(events):
    return [
    sum(e[3] for e in user_events if e[2] == "confirmation") > 0
    for user, user_events in groupby(events, lambda e: e[0])
    ]
exp_paying_users = amount_of_paying_users(exp_events)
control_paying_users = amount_of_paying_users(control_events)

In [305]:
def effect(a):
    return np.mean(a)
def statistic(a, b):
    return effect(a) - effect(b) 

result = permutation_test((exp_paying_users, control_paying_users), statistic, permutation_type="independent")

print("RESULTS for metric \"Amount of paying users\": ", end='')
print_result(result)

RESULTS for metric "Amount of paying users": 0.0002 red


Эффект является статистически значимым, поэтому возможно опровергнуть нулевую гипотезу.

Количество платящих пользователей действительно уменьшилось, о чем свидетельствует "red" в результатах теста.

## Выводы

По окончанию проверки двух гипотез удалось установить уменьшение количества платящих пользователей, но увеличение среднего чека от пользователя.

Поскольку $ARPU=\frac{Revenue}{P+NP}$, где $P$ – количество платящих пользователей, а $NP$ – количество неплатящих пользователей, и $ARPPU=\frac{Revenue}{P}$, где $P$ – количество платящих пользователей, то для ЛПР можно указать, что с увеличением числителя, увеличивается значение, а с увеличением знаменателя, значение уменьшается. 